## CNN - Dogs vs Cats

dataset: https://www.kaggle.com/competitions/dogs-vs-cats/data

In [1]:
# Biblioteki

import os
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Input, MaxPooling2D,Dropout,BatchNormalization,Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
# Constants

TRAIN_IMAGES_PATH = 'data/dogs-vs-cats/train'
TEST_IMAGES_PATH = 'data/dogs-vs-cats/test1'

IMAGE_LABEL = "image"
CLASS_LABEL = "class"
DOG_LABEL = "dog"
CAT_LABEL = "cat"

VALID_SIZE = 0.2

BATCH_SIZE = 32

IMG_WIDTH=128
IMG_HEIGHT=128
IMG_CHANNELS=3
IMG_SIZE=(IMG_WIDTH, IMG_HEIGHT)

KERNEL_SIZE = (3,3)
POOL_SIZE = 2
RELU_ACTIVATION = 'relu'

EPOCHS = 10 # pewnie można zmienić do 15 np.

### Preprocessing

In [ ]:

train_images = os.listdir(TRAIN_IMAGES_PATH)
test_images = os.listdir(TEST_IMAGES_PATH)

classes = [DOG_LABEL if DOG_LABEL in filename else CAT_LABEL for filename in train_images]
train_set_df = pd.DataFrame({IMAGE_LABEL: train_images, CLASS_LABEL: classes})

train_set_df.head()


,image,class
0,cat.0.jpg,cat
1,cat.1.jpg,cat
2,cat.10.jpg,cat
3,cat.100.jpg,cat
4,cat.1000.jpg,cat


In [ ]:
train_set_df[CLASS_LABEL].value_counts()

class
cat    12500
dog    12500
Name: count, dtype: int64

In [ ]:
# Podział na dane treningowe i walidacyjne:
train_df, val_df = train_test_split(train_set_df, test_size=VALID_SIZE, stratify=train_set_df[CLASS_LABEL], random_state=42)


In [ ]:
print("Train dataset:")
print(train_df[CLASS_LABEL].value_counts())
print("Validation dataset:")
print(val_df[CLASS_LABEL].value_counts())

Train dataset:
class
dog    10000
cat    10000
Name: count, dtype: int64
Validation dataset:
class
dog    2500
cat    2500
Name: count, dtype: int64


In [ ]:
# Edycja danych treningowych i walidacyjnych:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=(0.8, 1.5),
    horizontal_flip=True,
    brightness_range=(0.5, 1.5)
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# generators:

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=TRAIN_IMAGES_PATH,
    x_col=IMAGE_LABEL,
    y_col=CLASS_LABEL,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = valid_datagen.flow_from_dataframe(
    val_df,
    directory=TRAIN_IMAGES_PATH,
    x_col=IMAGE_LABEL,
    y_col=CLASS_LABEL,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


### Training

In [ ]:

model = Sequential([
    Conv2D(32, KERNEL_SIZE, activation=RELU_ACTIVATION, input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS)),
    BatchNormalization(),
    MaxPooling2D(POOL_SIZE,POOL_SIZE),
    Dropout(0.25),

    Conv2D(64, KERNEL_SIZE, activation=RELU_ACTIVATION),
    BatchNormalization(),
    MaxPooling2D(POOL_SIZE,POOL_SIZE),
    Dropout(0.25),

    Conv2D(128, KERNEL_SIZE, activation=RELU_ACTIVATION),
    BatchNormalization(),
    MaxPooling2D(POOL_SIZE,POOL_SIZE),
    Dropout(0.25),

    Conv2D(256, KERNEL_SIZE, activation=RELU_ACTIVATION),
    BatchNormalization(),
    MaxPooling2D(POOL_SIZE,POOL_SIZE),
    Dropout(0.25),

    Conv2D(512, KERNEL_SIZE, activation=RELU_ACTIVATION),
    BatchNormalization(),
    MaxPooling2D(POOL_SIZE,POOL_SIZE),

    Flatten(),

    Dense(1024, activation=RELU_ACTIVATION),
    Dense(512, activation=RELU_ACTIVATION),
    Dense(256, activation=RELU_ACTIVATION),
    Dense(128, activation=RELU_ACTIVATION),
    Dropout(0.5),

    Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

model.save("cats_dogs_cnn.h5")

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 414s 656ms/step - accuracy: 0.5700 - loss: 0.7421 - val_accuracy: 0.6496 - val_loss: 0.6634
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 443s 708ms/step - accuracy: 0.6937 - loss: 0.5932 - val_accuracy: 0.6396 - val_loss: 0.7381
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 402s 643ms/step - accuracy: 0.7199 - loss: 0.5482 - val_accuracy: 0.7290 - val_loss: 0.5785
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 451s 721ms/step - accuracy: 0.7638 - loss: 0.5027 - val_accuracy: 0.8006 - val_loss: 0.4600
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 486s 778ms/step - accuracy: 0.7910 - loss: 0.4561 - val_accuracy: 0.7668 - val_loss: 0.4849
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 551s 881ms/step - accuracy: 0.8243 - loss: 0.3937 - val_accuracy: 0.6806 - val_loss: 0.6178
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 573s 916ms/step - accuracy: 0.8412 - loss: 0.3581 - val_accuracy: 0.8852 - val_loss: 0.2796
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 442s 706ms/step - accuracy: 0.8564 -

### Testing

In [ ]:
test_file_names = os.listdir(TEST_IMAGES_PATH)
test_df = pd.DataFrame({IMAGE_LABEL: test_file_names})

test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df,
    TEST_IMAGES_PATH,
    x_col=IMAGE_LABEL,
    y_col=None,
    class_mode=None,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
predictions = model.predict(test_generator)

In [ ]:
sample_test = test_df.head(36)

plt.figure(figsize=(15,15))

for index, row in sample_test.iterrows():
    image_file = row[IMAGE_LABEL]
    image_class = row[CLASS_LABEL]
    path = os.path.join(TEST_IMAGES_PATH, image_file)
    img = load_img(path, target_size=IMG_SIZE)
    plt.subplot(6,6,index+1)
    plt.imshow(img)
    plt.title(f"{image_file}\n is {image_class}")
    plt.axis('off')
plt.tight_layout()
plt.show()